---
  title: "Ontologies in Java with OWL API"
  description: "Pratical introduction to ontologies with OWL API in Java."
  categories: 
    - Java
    - Lecture
    - Ontology
    - Jena
---

Dans le contexte informatique une ontologie est un modèle conceptuel d'un domaine métier. L'objectif de ce modèle est de faciliter le partage d'information à propos de ce domaine en s'appuyant sur des constructions standards. 

Les exemples les plus simples sont les taxonomies, comme par exemple en biologie la classification des êtres vivants. Plus généralement, une ontologie comprend aussi des contraintes à respecter ou des règles permettant d'inférer de nouveaux faits.  

Web Semantique : 
Berners-Lee, T., Hendler, J. & Lassila, O.  (2001).  "The Semantic Web," Scientific American, May 2001. 
Linked Open Data (LOD):

XSD
RDF
RDFS
OWL

https://www.w3.org/TR/2012/REC-owl2-primer-20121211/

Une ontologie décrite en utilisant OWL perlet des définir des classes, des propriétés, des individus et des valeurs. 
On distingue le niveau terminologique (TBox) qui décrit les connaissances générales d'un domaine. 
    "Il y a des département qui ont des employé et un directeur."
Le second niveau est factuel (ABox) il décrit représente une configuration précise 
    "Pierre est le directeur du département d'informatique. Marie est membre du département d'informatique."

In [1]:
%%loadFromPOM
<dependency>
 <groupId>ch.qos.logback</groupId>
 <artifactId>logback-classic</artifactId>
 <version>1.4.5</version>
</dependency>

<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>owlapi-distribution</artifactId>
    <version>5.5.0</version>
</dependency>
<dependency>
  <groupId>com.github.galigator.openllet</groupId>
  <artifactId>openllet-owlapi</artifactId>
  <version>2.6.5</version>
</dependency>

In [2]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.StringDocumentSource;
import org.semanticweb.owlapi.reasoner.OWLReasoner;

import openllet.owlapi.OpenlletReasoner;
import openllet.owlapi.OpenlletReasonerFactory;

import java.io.ByteArrayInputStream;

import ch.qos.logback.classic.util.ContextInitializer;

System.setProperty(ContextInitializer.CONFIG_FILE_PROPERTY, "/home/jovyan/work/javanotebook-full/logback.xml");

IJava.getKernelInstance().getMagics().registerCellMagic("loadOwlModel", (args, body) -> {
    body="""
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix : <http://ebruno.fr/2023/ontologies/test#> .    
"""+body;    
    OWLOntologyManager m = OWLManager.createOWLOntologyManager();
        OWLOntology owlOntology = m.loadOntologyFromOntologyDocument(new StringDocumentSource(body));
    
    final OpenlletReasoner reasoner = OpenlletReasonerFactory.getInstance().createReasoner(owlOntology);
    if (reasoner.isConsistent()) {
      reasoner.getKB().realize();
      reasoner.getKB().printClassTree();
    } else
        System.err.println("The model is not consistent.");
    
    StringBuilder out=new StringBuilder("Model loaded");
 display(out,"text/markdown");
 return out;
});

In [3]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
    :Mary a :Person .
    :Person a owl:Class .


 owl:Thing
    test:Person - (test:Mary)



Model loaded

## Les classes
### Classes énumérées

In [4]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Status rdf:type owl:Class ;
    owl:oneOf (:OnLine :Offline) .


 owl:Thing
    test:Status - (test:Offline, test:OnLine)



Model loaded

### Union

In [5]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:LegalAgent a owl:Class ;
owl:equivalentClass [
        a owl:Class ;
        owl:unionOf ( :Person :Group )
    ] .


 owl:Thing
    test:LegalAgent
       test:Group
       test:Person



Model loaded

### Intersection

In [6]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Man a owl:Class ;
    owl:equivalentClass [
        a owl:Class ;
        owl:intersectionOf ( :Person :Male )
    ].


 owl:Thing
    test:Male
       test:Man
    test:Person
       test:Man



Model loaded

### Négation

In [7]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
:Inedible a owl:Class ;
owl:equivalentClass [
  a owl:Class ;
  owl:complementOf :Edible
  ] .
    
:Pierre a :Edible.
:Mary :hasName "mary".


 owl:Thing
    test:Edible - (test:Pierre)
    test:Inedible



Model loaded

### Disjonction

deux classes : Une ressource ne peut pas appartenir aux deux classes en même temps.

In [8]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:Square rdf:type owl:Class ;
        owl:disjointWith :Circle .
    
:c a :Square ; 
   a :Circle . 

The model is not consistent.


Model loaded

In [9]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .
                   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
                   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
                   @prefix owl: <http://www.w3.org/2002/07/owl#> .
                   @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
                   :Mary a :Student; a :Professor .
                   :Student a owl:Class .
                   :Professor a owl:Class .
                       
                   :Student rdf:type owl:Class ;
                       owl:disjointWith :Professor .

The model is not consistent.


Model loaded

### Disjonction de plusieurs classes

une ressource ne peut, au plus, appartenir qu’à une seule de ces classes

In [10]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

[] rdf:type owl:AllDisjointClasses ;
   owl:members ( :Square :Circle :Triangle ) .
           
:c rdf:type :Square .
:c rdf:type :Circle .

The model is not consistent.


Model loaded

### Union disjointe
diviser une classe en une partition complète de sous classes

In [11]:
%%loadOwlModel

:Passenger rdf:type owl:Class ;
           owl:disjointUnionOf :Adult, :Child, :Pet .
               
:p1 a :Adult ;             
    a :Child ;
    a :Pet .

13:20:40 WARN - Possible malformed list: rdf:first triple missing


EvalException: Exception occurred while running cell magic 'loadOwlModel': classExpressions cannot be null or empty

In [12]:
%%loadOwlModel

:John a :Man ;
    :name "John" ;
    :hasSpouse :Mary .
:Mary a :Woman ;
    :name "Mary" ;
    :hasSpouse :John .
:John_jr a :Man ;
    :name "John Jr." ;
    :hasParent :John, :Mary .
:Time_Span a owl:Class .
:event a :Activity ;
    :has_time_span [
        a :Time_Span ;
        :at_some_time_within_date "2018-01-12"^^xsd:date
        ] .
:u129u-klejkajo-2309124u-sajfl a :Person ;
    :name "John Doe" .


 owl:Thing
    test:Activity - (test:event)
    test:Man - (test:John_jr, test:John)
    test:Person - (test:u129u-klejkajo-2309124u-sajfl)
    test:Time_Span - (1 Anonymous Individual)
    test:Woman - (test:Mary)



Model loaded

## Les propriétés

  1. owl:ObjectProperty sont des relations entre des ressources
  1. owl:DatatypeProperty ont des valeurs littérales (typées)
  1. owl:AnnotationProperty sont ignorées dans les inférences et utilisées pour documenter ou pour des extensions

Une relation peut être symétriques ou non, et/ou réflexive ou non.

AsymmetricProperty

In [13]:
%%loadOwlModel

:hasSpouse rdf:type owl:ObjectProperty ,
        owl:SymmetricProperty ,
        owl:IrreflexiveProperty .


 owl:Thing



Model loaded

In [14]:
%%loadOwlModel      

:x :hasSpouse :y.
:b :hasSpouse :b.


 owl:Thing



Model loaded

inverse : deux relations qui existent simultanément en sens inverse (ex. parent_de /
enfant_de)

transitives : une relation qui se propage de proche en proche (ex. Tom ancêtre Jim ancêtre
Jules)

propriétés disjointes : Des relations qui ne peuvent pas exister en même temps sur le même sujet et le même objet

In [15]:
%%loadOwlModel

:hasAncestor rdf:type owl:ObjectProperty ,
        owl:transitive ;
        owl:inverseOf :hasDescendant .            

:hasSon rdf:type owl:ObjectProperty ;
        owl:disjointwith :hasDaughter.             
            
:Mary :hasHusband :Pierre .
:Mary :hasAncestor :Jean .
:Jean :hasAncestor :Sophie .
            

13:20:41 INFO - Unparsed triple: http://www.w3.org/2002/07/owl#disjointwith -> http://ebruno.fr/2023/ontologies/test#hasSon -> http://ebruno.fr/2023/ontologies/test#hasDaughter



 owl:Thing - (test:Jean, test:Mary, test:Sophie)
    owl:transitive - (test:hasAncestor)



Model loaded

In [16]:
%%loadOwlModel
          
:hasSon rdf:type owl:ObjectProperty ;
        owl:propertyDisjointWith :hasDaughter.
            
:x :hasSon :y .
:x :hasDaughter :y .

13:20:41 INFO - Unparsed triple: http://www.w3.org/2002/07/owl#propertyDisjointWith -> http://ebruno.fr/2023/ontologies/test#hasSon -> http://ebruno.fr/2023/ontologies/test#hasDaughter



 owl:Thing - (test:x, test:y)



Model loaded

propriétés chaînées
des relations qui mises bout à bout impliquent une autre relation
(ex. parent + frère = oncle)

In [17]:
%%loadOwlModel

:hasUncle owl:propertyChainAxiom (:hasParent :hasBrother).

:hasGrandparent owl:propertyChainAxiom ( :hasParent :hasParent ).

:x :hasParent :y.
:y :hasParent :z.
:y :hasBrother :y1.


 owl:Thing



Model loaded

propriétés fonctionnelles
une relation pour laquelle une ressource ne peut avoir qu’une valeur (ex.
naissance)

propriétés inverses fonctionnelles
une relation pour laquelle une même valeur implique la même ressource (ex.
NSS)

In [18]:
%%loadOwlModel
@prefix f: <http://example.com/owl/families#> .
@prefix g: <http://example.com/owl/families#> .

f:hasWife rdf:type owl:ObjectProperty ,
        owl:InverseFunctionalProperty ,
        owl:FunctionalProperty ,
        owl:AsymmetricProperty ,
        owl:IrreflexiveProperty ;
    rdfs:domain f:Person ;
    rdfs:range f:Person ;
    rdfs:subPropertyOf f:loves ;
    rdfs:range f:Woman ;
    rdfs:subPropertyOf f:hasSpouse ;
    rdfs:domain f:Man .
        
:x f:hasWife :y .


 owl:Thing
    families:Man - (test:x)
    families:Person - (test:x, test:y)
    families:Woman - (test:y)



Model loaded

identification par des clés
deux instances qui ont même(s) valeur(s) de clé(s) sont en réalité la même
instance

In [19]:
%%loadOwlModel
@prefix : <http://ebruno.fr/2023/ontologies/test#> .

:Person owl:hasKey ( :hasSSN ) .
    
:x :hasSSN "1234" ; 
   :hasName "Pierre" .
:y :hasSSN "1234" .


13:20:41 WARN - Property http://ebruno.fr/2023/ontologies/test#hasSSN is undeclared at this point in parsing: typing as OWLDataProperty



 owl:Thing
    test:Person



Model loaded